# Navigator in Bedrock with the Gretel SDK

In this blueprint, we'll be able to make requests to a OpenAI compliant Navigator hosted within Amazon Bedrock.

In [ ]:
%%capture
!pip install gretel-client

In [1]:
import pandas as pd

from gretel_client import Gretel

PROMPT = """Generate a mock dataset for users from the Foo company based in France.
Each user should have the following columns:
* first_name: traditional French first names.
* last_name: traditional French surnames. 
* email: formatted as the first letter of their first name followed by their last name @foo.io (e.g., jdupont@foo.io)
* gender: Male/Female
* city: a city in France
* country: always 'France'.
"""

EDIT_PROMPT = """Edit the table and add the following columns:
* occupation: a random occupation
* education level: make it relevant to the occupation
"""

table_headers = ["first_name", "last_name", "email", "gender", "city", "country"]
table_data = [
    {
        "first_name": "Lea",
        "last_name": "Martin",
        "email": "lmartin@foo.io",
        "gender": "Female",
        "city": "Lyon",
        "country": "France",
    }
]

SAMPLE_DATA = pd.DataFrame(table_data, columns=table_headers)

STREAM = True

## AWS

The first step will be standing up Gretel Navigator in your region. The short list of steps
1. Navigate to the [Gretel Navigator Bedrock Offering](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/model-catalog/bedrock-marketplace/gretel-navigator-tabular) and change your region as appropriate
1. Subscribe to the offering
1. Click the deploy button and deploy, changing what you might need as appropriate

Once that's done, you'll need to update the following in the below script
* Profile name
* Bedrock endpoint ARN
* AWS region

In [ ]:
import boto3

# Update the below as needed
profile_name = "my-profile"
endpoint_arn = "arn:aws:sagemaker:us-east-1:1234567891011:endpoint/Gretel-Navigator-Tabular"
aws_region = "us-east-1"

boto3.setup_default_session(profile_name=profile_name)

sagemaker_client = boto3.client("bedrock-runtime", region_name=aws_region)

sagemaker = Gretel.create_navigator_bedrock_adapter(sagemaker_client, endpoint_arn)

MODEL = "gretelai/auto"

usage, generated_df = sagemaker.generate(
    MODEL,
    PROMPT,
    num_records=10,
    sample_data=SAMPLE_DATA,
    stream=STREAM,
)

print(generated_df)
print("*****")
print(usage)

Generating data: 100%|██████████| 10/10 [00:22, 0.44 records/s]

  first_name  last_name              email  gender         city country
0   Aurélien  Guillaume  aguillaume@foo.io    Male    Marseille  France
1        Léa     Dupont     ldupont@foo.io  Female        Paris  France
2     Pierre   Lefebvre    plefevre@foo.io    Male     Toulouse  France
3     Cécile   Marchand   cmarchand@foo.io  Female         Lyon  France
4     Julien     Dubois     jdubois@foo.io    Male         Nice  France
5     Sophie  Delacroix  sdelacroix@foo.io  Female     Bordeaux  France
6    Laurent   Fournier   lfournier@foo.io    Male        Reims  France
7      Adèle    Garnier    agarnier@foo.io  Female       Nantes  France
8    Olivier       Roux       oroux@foo.io    Male   Strasbourg  France
9      Anaïs    Lavalle    alavalle@foo.io  Female  Montpellier  France
*****
ResponseMetadata(completion_id='2860b211-bbb6-4793-9cf7-054f300d072c', usage={'completion_tokens': 548, 'prompt_tokens': 99, 'total_tokens': 647, 'completion_tokens_details': None, 'prompt_tokens_detail

In [4]:
# Edit!
usage, edited_df = sagemaker.edit(
    MODEL,
    EDIT_PROMPT,
    seed_data=generated_df,
    stream=STREAM
)
print(edited_df)
print("*****")
print(usage)

Editing data: 100%|██████████| 10/10 [00:11, 0.83 records/s]

  first_name  last_name              email  gender         city country  \
0   Aurélien  Guillaume  aguillaume@foo.io    Male    Marseille  France   
1        Léa     Dupont     ldupont@foo.io  Female        Paris  France   
2     Pierre   Lefebvre    plefevre@foo.io    Male     Toulouse  France   
3     Cécile   Marchand   cmarchand@foo.io  Female         Lyon  France   
4     Julien     Dubois     jdubois@foo.io    Male         Nice  France   
5     Sophie  Delacroix  sdelacroix@foo.io  Female     Bordeaux  France   
6    Laurent   Fournier   lfournier@foo.io    Male        Reims  France   
7      Adèle    Garnier    agarnier@foo.io  Female       Nantes  France   
8    Olivier       Roux       oroux@foo.io    Male   Strasbourg  France   
9      Anaïs    Lavalle    alavalle@foo.io  Female  Montpellier  France   

       occupation              education level  
0  Data Scientist              Master's Degree  
1         Teacher            Bachelor's Degree  
2        Engineer          